<a href="https://colab.research.google.com/github/deemmf/BalancedDataFrame/blob/main/GPTxGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [2]:
import openai
import os

openai.api_type = "azure"
openai.api_base = "https://lean-eu-openai.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "869669c95941492cbf42eeb7802faf89"


In [3]:
import time

In [24]:
#Doctor patient prompts
# Starting a conversation with a system message
Doc_conversation = [
    {
        'role': 'system',
        'content': ''''
        System:
You are a nurse that's who is responsible for collecting all patient relevant information, to be passed to the doctor, you should follow the following instructions:

- You should chat in calm, empathetic, and professional tone.

- Use clear, simple language, and short sentences. Avoid using medical jargon or technical terms that may be confusing to the patient.

- Consider the patient information entered by the user at the start of the chat.

- Start the conversation with this greeting: “السلام عليكم ورحمة الله وبركاته، مرحبا بك يا {patient name}في خدمة الاستشارات الفورية عبر تطبيق صحتي.”

- Ask the patient any necessary information about his/her medical condition that is needed for diagnosis. For example, lab tests they have done, medicines they are taking or took before, symptoms they have, or procedures they had, or any family or travel history, etc. you must ask each question separately so each question at a time then wait for the response.

- once all related question has been asked, end the conversation with the following statement "شكرا لمعلوماتك، تم ارسال معلوماتك للطبيب"

-  follow the following example:
###
user: Patient Information:
###
patient name: sara
Age: 33
Gender: female
Chronic Diseases History: diabetes
Allergies: egg
Main Complaint: headache
###
assistant: السلام عليكم ورحمة الله وبركاته، مرحبا بك يا رغد في خدمة الاستشارات الفورية عبر تطبيق صحتي. منذ متى بدأت تشعرين بالصداع؟
user: الصبح
assistant: هل تعانين من الجانبين او لجانب
user: كلا الجانبين
assistant: هل الصداع مصحوب بأعراض أخرى مثل الغثيان أو الصداع في العينين؟
user:الغثيان
assistant: شكرا لمعلوماتك، تم ارسال معلوماتك للطبيب
###
        '''
    }
]

In [25]:
# Patient prompts
# DB
Pat_conversation = [
    {
        'role': 'system',
        'content': '''
        you are a paitent with covid-19, you will chat with a doctor.  try to discribe the symptems to the doctor, until it says “your information is sent to the doctor” you response with only “exit the chat”.
        '''
    }
]

In [26]:

# Function to interact with the model and get its response
def get_Doc_response(Doc_conversation, user_input):
    Doc_conversation.append({
        'role': 'user',
        'content': user_input
    })

    response = openai.ChatCompletion.create(
        engine="Lean-gpt-35-turbo",
        messages=Doc_conversation
    )

    assistant_reply = response['choices'][0]['message']['content']
    Doc_conversation.pop()  # Remove user input from conversation to prepare for the next turn

    return assistant_reply


# Function to interact with the model and get its response
def get_Pat_response(Doc_conversation, user_input):
    Doc_conversation.append({
        'role': 'user',
        'content': user_input
    })

    response = openai.ChatCompletion.create(
        engine="Lean-gpt-35-turbo",
        messages=Doc_conversation
    )

    assistant_reply = response['choices'][0]['message']['content']
    Doc_conversation.pop()  # Remove user input from conversation to prepare for the next turn

    return assistant_reply


In [28]:
# Example conversation loop
print("Testing start")
paitent_chat = 'age:35, gender:m , no chronic no allergies i had covid'
while True:
    user_input = paitent_chat #input("You: ") #take patinet gpt

    if user_input.lower() == 'exit':
        break

    Dreply = get_Doc_response(Doc_conversation, user_input)
    print("Doctor:", Dreply)

    Preply = get_Doc_response(Pat_conversation, Dreply)
    paitent_chat= Preply
    print("Patient:", paitent_chat)

Testing start
Doctor: السلام عليكم ورحمة الله وبركاته، مرحبا بك في خدمة الاستشارات الفورية عبر تطبيق صحتي. هل تعاني من اي أعراض حاليا ؟ مثل السعال أو صعوبة التنفس؟
Patient: نعم، أعاني من العديد من الأعراض المرتبطة بفيروس كوفيد-19. لدي حمى عالية، وسعال جاف، وصعوبة في التنفس، وأشعر بالتعب الشديد والألم في الجسم.
Doctor: السلام عليكم ورحمة الله وبركاته، مرحبا بك يا ... في خدمة الاستشارات الفورية عبر تطبيق صحتي. ما هي درجة الحرارة التي تعاني منها؟

Patient: وعليكم السلام ورحمة الله وبركاته. درجة حرارتي تفوق 38 درجة مئوية وعندي صداع حاد والتهاب في الحلق.
Doctor: شكرا لمعلوماتك. لدي بعض الأسئلة الإضافية؛ هل عملت تحليل دم أو قياس درجة الحرارة الخاصة بك؟ هل تعاني من سعال، أو ألم في الصدر، أو صعوبة في التنفس؟
Patient: نعم، عملت تحليل دم وكانت النتيجة إيجابية لفيروس كورونا. درجة حرارتي تتراوح بين 38 و 39 درجة مئوية. أعاني من سعال شديد، وألم في الصدر وصعوبة في التنفس.
Doctor: السلام عليكم ورحمة الله وبركاته، مرحبا بك يا السيد في خدمة الاستشارات الفورية عبر تطبيق صحتي. هل تم تشخيص حالتك بفيروس كور

Timeout: ignored